In [1]:
''' Additional Functions '''
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools
from itertools import islice
from re import sub, search
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import os

# nicer print variant
def pprint(dlist):
    ldf = pd.DataFrame(dlist)
    display(ldf)

In [2]:
!pip install /kaggle/input/biopylib/biopylib-0.0.7-py3-none-any.whl

Processing /kaggle/input/biopylib/biopylib-0.0.7-py3-none-any.whl


In [3]:
# import sequence related classes 
from biopylib.sequence import SQ
from biopylib.read_sequence import readSeq
from biopylib.sequence_alignment import SUBM,ALI,PSA,MSA

![](https://i.imgur.com/MTR8R2d.jpg)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>1 |</span></b> <b>BACKGROUND</b></div>

### <b><span style='color:#E888BB'> 1.1 |</span> BIOPYLIB </b>

- In this notebook, we'll introduce a few **<span style='color:#E888BB'>custom classes</span>** that will enable us to **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">align</mark>** & compare multiple **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">biological sequences</mark>**; a common task in **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">bioinformatics</mark>**
- And whilst it's quite handy to already to preexisting libraries & it is worthwile to make our own libraries that will be to our liking, the end product of course being **[a library](https://www.kaggle.com/shtrausslearning/bioseq)** we can use and update
- Nevertheless, in **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Section 7</mark>**, we've added sequence alignment related operations in  **<span style='color:#E888BB'>biopython</span>** as well, which is quite handy because we don't need to fiddle with any internal code

This notebook is a continuation from **[Biological Sequence Operations](https://www.kaggle.com/shtrausslearning/biological-sequence-operations)**, so it's better to read that one first as we need functions from <code>bioseq-1.0-py3-none-any.whl</code>:
> - We'll be needing <code>read_seq</code> to read any FASTA files containing sequences
> - As well as the main sequence operations class <code>SQ</code>, as the classes introduces here will require it

### <b><span style='color:#E888BB'> 1.2 |</span> Cell Information </b>

Cells contain molecules which will reference to througout the notebook: **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">amino acids</mark>**, **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">nucleotides</mark>**, **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">proteins</mark>**

A **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">cell</mark>** is mostly composed of water:
> - a bacteria cell has a weight composition of roughly 70% water & 30% <b>chemical origin</b>, 
> - <b>7% -> small molecules</b> Inc. **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">amino acids</mark>** and **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">nucleotides</mark>**
> - 23% -> <b>macro molecules</b> (**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">proteins</mark>**,lipids,polysaccharides)



According to their internal structure, they can be divided into to major categories:

> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Prokaryotic</mark>** cells : have no nucleus or internal membranes
> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Eukaryotic</mark>** cells : which have a defined <b>nucleus</b>, <b>internal membranes</b> and functional elements called <b>organelles</b>.


- At a structural level, all cells are surrounded by a structure called cell <b>membrane</b> or <b>plasma membrane</b>. 
- This <b>membrane</b> is permeable to molecules that cells need to absorb from or excrete to the outside medium.
- Within the cell we find the <b>cytoplasm</b> (largely composed of water), which serves as the medium for the cell

### <b><span style='color:#E888BB'> 1.3 |</span> DNA Strands </b>

#### <b><span style='color:#E888BB'>COMPLEMENTARY STRANDS IN DNA</span></b>

- DNA is a molecule composed of **<span style='color:#E888BB'>two complementary strands</span>** that form and stick together due to the connections established between the nucleotides in both strands
- This is made possible by due to the chemical phenomenon:
    - Where **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Adenine (A)</mark>** bonds only with **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Thymine (T)</mark>** nucleotides; as a result of **two hydrogen connections**
    - Similarly, **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Guanine (G)</mark>** bonds only with **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Cytosine (C)</mark>** nucleotides by three hydrogen connections

#### <b><span style='color:#E888BB'>REVERSE COMPLEMENT</span></b>

- This results in **two complementary** and **anti-parallel strands** (connected in opposite directions), if we know the nucleotide sequence in one of the strands, we can get the sequence in the opposite strand by taking the complement of its nucleotides, which are also read backwards, thus we have the **reverse complement** of the other strand.
- It has become a **standard to describe the DNA though only one** of the strands, due to this **complementarity** using <b>[A,T,G,C]</b>.
- The existence of these two strands is essential in order to **pass on genetic information** to new cells and **produce proteins**.

### <b><span style='color:#E888BB'> 1.4 |</span> Sequence Alphabets </b>

**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ABC (I/II)</mark>** **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Nucleic Acids</mark>**

> - Among <b>molecules with a biological role</b>, we can find **<span style='color:#E888BB'>nucleic acids</span>**
> - Nucleic acids encode and express the genetic code that is kept within the cell
- There are two major types of **<span style='color:#E888BB'>nucleic acids</span>**: 
> - **<span style='color:#E888BB'>Deoxyribo Nucleic Acid (DNA)</span>**
> - **<span style='color:#E888BB'>Ribonucleic Acid (RNA)</span>** (Obtainable via transcription)
- DNA contains the information necessary to build a cell, and keep it functioning. 
- In <b>eukaryotic</b> cells, DNA will be found in the nucleus, whilst in the <b>prokaryotic</b> cells, it will be found in the cytoplasm. 
- <b>IUPAC</b> defines the full list of nucleotides as shown in the table below, with <b>A,T,G,C</b> being the main four:
- Another type of nucleotide list often used is **[IUB Ambiguity Codes](http://biocorp.ca/IUB.php)**, which we use later in the notebook as well

**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ABC (II/II)</mark>** **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Amino Acids</mark>**

**<span style='color:#E888BB'>Amino acids</span>**:
> The **building blocks of proteins**, which are <b>macromolecules</b> that perform most of the functions inside a cell

Proteins have a **broad range of functions**, spanning from **catalytic** to **structural functions**:

> - **<span style='color:#E888BB'>Enzymes</span>** : Type of abundant proteins that promote chemical reactions & convert some molecules into other types of molecules required for the functioning of the cell
> - **<span style='color:#E888BB'>Carbohydrates</span>** : Serve as energy storage, both for immediate and long term energy demands
> - **<span style='color:#E888BB'>Lipids</span>** : Part of the plasma membrane, doing signaling and energy storage

The cell also contains other components of varying complexity. Of importance: 
> - <b>Mitochondria</b> & the <b>Chloroplasts</b> : Organelles involved in the production of energy. 
> - <b>Ribosomes</b> : Large and complex molecules composed of a mixture of genetic material, req. to assemble proteins and play a central role in the flow of genetic information

### <b><span style='color:#E888BB'> 1.5 |</span> Biological Sequences </b>

- A **<span style='color:#E888BB'>biological sequence</span>** represents a single, continuous molecules of **<span style='color:#E888BB'>nucleic acid (nucleotide)</span>** or **<span style='color:#E888BB'>amino acids</span>**
- Subsequently we can also **<span style='color:#E888BB'>compare multiple sequences</span>** based on some form of alignment methodology, so that's what this notebook is about
    
**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Sequence Alignment Introduction</mark>** | **[Data Mining Trends and Research Frontiers](https://www.sciencedirect.com/topics/computer-science/biological-sequence)**

<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
    
<ul>
    <li>Sequence alignment is based on the fact that all living organisms are related by evolution.</li>
    <li>This implies that the nucleotide (DNA, RNA) and protein sequences of species that are closer to each other in evolution should exhibit more similarities.</li>
    <li>An alignment is the process of lining up sequences to achieve a maximal identity level, which also expresses the degree of similarity between sequences.</li>
    <li>Two sequences are homologous if they share a common ancestor. The degree of similarity obtained by sequence alignment can be useful in determining the possibility of homology between two sequences</li>
    <li>Such an alignment also helps determine the relative positions of multiple species in an evolution tree, which is called a phylogenetic tree.</li>
    </ul>
    
</div> 

### <b><span style='color:#E888BB'> 1.6 |</span> Bio Sequence Examples </b>

In notebook **[Biological Sequence Operations](https://www.kaggle.com/shtrausslearning/biological-sequence-operations)**, we looked at how to read biological sequence files, here are two examples of what they might look like:

- An example of a **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Nucleotide</mark>** Sequence:

ref|NC_005816.1|:4343-4780 pesticin immunity protein [Yersinia pestis biovar Microtus str. 91001]
> ATGGGAGGGGGAATGATCTCAAAGTTATTTTGCTTGGCTCTCATATTTTTATCATCAAGTGGCCTTGCAG
AAAAAAACACATATACAGCAAAAGACATCTTGCAAAACCTAGAATTAAATACCTTTGGCAATTCATTGTC
TCATGGCATCTATGGGAAACAGACAACCTTCAAGCAAACCGAGTTTACAAATATTAAAAGCAACACCAAA
AAACACATTGCACTTATCAATAAAGACAACTCATGGATGATATCATTAAAAATACTAGGAATTAAGAGAG
ATGAGTATACTGTCTGTTTTGAAGATTTCTCTCTAATAAGACCGCCAACATATGTAGCCATACATCCTCT
ACTTATAAAAAAAGTAAAATCTGGAAACTTTATAGTAGTGAAAGAAATAAAGAAATCTATCCCTGGTTGC
ACTGTATATTATCATTAA

- An example of an **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Amino Acid</mark>** Sequence:

gi|7525080|ref|NP_051037.1| ribosomal protein S12 [Arabidopsis thaliana]
> MPTIKQLIRNTRQPIRNVTKSPALRGCPQRRGTCTRVYTITPKKPNSALRKVARVRLTSGFEITAYIPGI
GHNLQEHSVVLVRGGRVKDLPGVRYHIVRGTLDAVGVKDRQQGRSKYGVKKPK

### <b><span style='color:#E888BB'> 1.7 |</span> Homology & Similarity </b>

- Some key terms in **biological sequence alignment** : **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Homology</mark>** & **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Similarity</mark>**, we probably want to familiarise ourselves with them first


First things first, we have two sequences, **<span style='color:#E888BB'>DNA</span>** sequences to be exact (here they are from file **<span style='color:#E888BB'>NC_005816.1</span>**):

#### <b><span style='color:#E888BB'>SELECT TWO SEQUENCES</span></b>

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Sequence #1</mark>**

ref|NC_005816.1|:c8360-8088 hypothetical protein YP_pPCP10 [Yersinia pestis biovar Microtus str. 91001]

> TTGGCTGATTTGAAAAAGCTACAGGTTTACGGACCTGAGTTACCCAGGCCATATGCCGATACCGTGAAAG
GTTCTCGGTACAAAAATATGAAAGAGCTTCGCGTTCAGTTTTCTGGCCGTCCGATAAGAGCCTTTTATGC
GTTCGATCCGATTCGTCGGGCTATCGTTCTTTGTGCAGGAGATAAAAGTAATGATAAGCGGTTTTATGAA
AAACTGGTGCGTATAGCTGAGGATGAATTTACAGCACATCTGAACACACTGGAGAGCAAGTAA

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Sequence #2</mark>**

ref|NC_005816.1|:c8088-7789 putative transcriptional regulator [Yersinia pestis biovar Microtus str. 91001]

> ATGAGAACATTAGATGAGGTGATTGCCAGTCGTTCACCTGAAAGCCAGACACGAATTAAAGAAATGGCAG
ATGAGATGATTCTTGAGGTCGGCTTGCAGATGATGCGTGAAGAACTCCAGTTATCACAAAAACAAGTTGC
TGAGGCGATGGGTATAAGCCAGCCAGCAGTAACAAAGCTGGAGCAGCGCGGAAATGATTTAAAGCTGGCG
ACGTTAAAGCGTTACGTTGAAGCAATGGGAGGCAAATTAAGCTTGGATGTTGAGCTTCCTACAGGAAGGA
GAGTAGCGTTCCATGTCTAA

#### <b><span style='color:#E888BB'>HOW TO TELL IF TWO SEQUENCES ARE HOMOLOGOUS?</span></b>

**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">HOMOLOGOUS</mark>** | **[Wikipedia](https://en.wikipedia.org/wiki/Homology_(biology)**
> - Similar biological structures or sequences in different taxa are homologous if they are derived from a common ancestor
> - Thus, two sequences are said to be **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">homologous</mark>** if they are both derived from a **<span style='color:#E888BB'>common ancestral sequence</span>**

#### <b><span style='color:#E888BB'>HOW SIMILAR ARE THESE SEQUENCES?</span></b>

If we wanted to know how **closely they are related**, we could **<span style='color:#E888BB'>make some assumptions</span>** about their relation to one another:
> - We are assuming that there exists an ancestry relation between the two sequences, which is based on this **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">sequence similarity</mark>**
> - We can **<span style='color:#E888BB'>generate a hypothesis</span>** about the biological function of a sequence we are exploring, based on its **<span style='color:#E888BB'>similarity to another sequence</span>**, which already has a determined function
> - Sequences that display a significant **<span style='color:#E888BB'>degree of similarity</span>** have a high probability of being **<span style='color:#E888BB'>homologous</span>** and **<span style='color:#E888BB'>sharing similar functions</span>**
> - Two sequences that have a high order of similarity -> have a high probability of being homologous
> - There doesn't seem to exist a point at which this becomes a certainty & thus requires experimental verification
> - The **<span style='color:#E888BB'>higher the degree of similarity</span>**, the more confident we can be that **<span style='color:#E888BB'>two sequences are homologous</span>** & thus share similar functions

### <b><span style='color:#E888BB'> 1.8 |</span> Biological Mutations </b>

So what stops us from just placing two sequences next to eachother & comparing them; there exist some factors that do add an extra level of complication to our similarity assumption:

- Biological **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">mutations</mark>** signifcantly complicate the process of what may seem quite a straightforward task
- **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Insertions</mark>** & **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">deletions</mark>**, as well as **<span style='color:#E888BB'>changes in multiple nucleotides</span>** due to these mutations can make the comparison of sequences more complex

**<mark style="background-color:#3b3745;color:white;border-radius:5px;opacity:0.9">MUTATIONS</mark>** | **[GENOME.GOV](https://www.genome.gov/genetics-glossary/Mutation)**

<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
    
<ul>
    <li>A <b><span style='color:#E888BB'>mutation</span></b> is a change in a DNA sequence.</li>
    <li>Mutations can result from DNA copying mistakes made during cell division, <br>
        exposure to ionizing radiation, exposure to chemicals called mutagens, or infection by viruses.</li>
    <li>Germ line mutations occur in the eggs and sperm and can be passed on to offspring, while <br>
        somatic mutations occur in body cells and are not passed on.</li>
    </ul>

</div>

<br>
    
**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">INSERTIONS & DELETIONS</mark>** | **[NATURE](https://www.nature.com/scitable/topicpage/dna-is-constantly-changing-through-the-process-6524898/)**

<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
    
<ul>
    <li><b>Insertions</b> and <b>deletions</b> are two other types of mutations that can affect cells at the gene level.</li>
    <li>An insertion mutation occurs when an extra nucleotide is added to the DNA strand during replication. </li>
    <li>This can happen when the replicating strand "slips," or wrinkles, which allows the extra nucleotide to be incorporated</li>
    <li><b>Strand slippage</b> can also lead to deletion mutations.</li>
    <li>A deletion mutation occurs when a wrinkle forms on the DNA template strand and subsequently causes a nucleotide to be omitted from the replicated strand.</li>
    </ul>
</div>

### <b><span style='color:#E888BB'> 1.9 |</span> Comparing Sequences </b>

#### <b><span style='color:#E888BB'>SEQUENCE ALIGNMENT ALGORITHMS</span></b>

The process of determining how similar two sequences are, taking into account biological **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">mutations</mark>**:

- Is referred to as <b><span style='color:#E888BB'>sequence alignment</span></b> & its subsequent **<span style='color:#E888BB'>algorithms</span>** are developed to achieve this
- The aim of such algorithms is to find <b>shared individual characters</b> in <b>both sequences</b>, <b><span style='color:#E888BB'>without changing the order</b>
- <b><span style='color:#E888BB'>gaps</span></b> | We will introduce <b><span style='color:#E888BB'>gaps</span></b> (spacing characters '_' or "-" ) : In an attempt to <b><span style='color:#E888BB'>maximise the characters shared between two sequences</span></b>
- Sequences with a high degree of similarity: Have a large number of <b>identical characters</b> (**nucleotides** or **aminoacids**) 
- There are two main classes of <b><span style='color:#E888BB'>sequence alignment</span></b>: **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">global sequence alignment</mark>** **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">local sequence alignment</mark>**

### <b><span style='color:#E888BB'> 1.10 |</span> Object Fuctions </b>

#### <b><span style='color:#E888BB'>OPTIMISATION PROBLEM</span></b>

An important part of sequence alignment algorithms:

- Sequence alignment is treated as an <b><span style='color:#E888BB'>optimisation problem</span></b>, with a defined <b>objective function</b>
- The <b><span style='color:#E888BB'>objective function</span></b> in the optimisation problem is used to <b><span style='color:#E888BB'>evaluate all feasible solutions</span></b> for the problem
- Assigning to each feasible solution (**all possible pairs**) a value & <b><span style='color:#E888BB'>finding the solution that maximises this function</span></b>; <b>score</b>

#### <b><span style='color:#E888BB'>COMPONENTS OF THE ALGORITHM</span></b>

In short, the **objective function** -> <b><span style='color:#E888BB'>substitution matrix</span></b> & <b><span style='color:#E888BB'>gap penalty</span></b> model:
> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Substitution Matrix</mark>** - Matrix which includes score values for all combinations of alignment where there are no gaps present in the comparison of two characters
> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Gap Penalty Model</mark>** - Model which defines how to penalise the case where we have no matches & the alignment has a gap in one of the sequences

### <b><span style='color:#E888BB'> 1.11 |</span> Sequence alignment uses </b>

#### <b><span style='color:#E888BB'>SEQUENCE ANNOTATION & PHYLOGENETIC ANALYSIS</span></b>

- Both **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">DNA</mark>** & **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">protein</mark>** sequence sequence alignment can be used for <b>sequence annotation</b>; **<span style='color:#E888BB'>giving functions to sequences</span>** (or parts of it)

**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">MSA: algorithms and applications</mark>** | **[@NCBI](https://pubmed.ncbi.nlm.nih.gov/10463075/)**

<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
    
Multiple sequence alignment has been proven to be a powerful tool for many fields of studies; 
Such as <b>phylogenetic reconstruction</b>, illumination of functionally important regions, and prediction of higher order structures of proteins and RNAs

</div>
      
- Such an alignment also helps determine the relative positions of multiple species in **<span style='color:#E888BB'>an evolution tree</span>**, which is called a **<span style='color:#E888BB'>phylogenetic tree</span>** (in **<span style='color:#E888BB'>phylogenetic analysis</span>**)
- An example notebook about **[phylogenetic trees](https://www.kaggle.com/nwheeler443/understanding-phylogenetic-trees)**

#### <b><span style='color:#E888BB'>RELEVANT REFERENCES</span></b>

- **[Needleman-Wunsch algorithm for sequence alignment](https://www.cs.sjsu.edu/~aid/cs152/NeedlemanWunsch.pdf)**
- **[BLAST Global Sequence Alignment](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&PROG_DEF=blastn&BLAST_PROG_DEF=blastn&BLAST_SPEC=GlobalAln&LINK_LOC=BlastHomeLink)**
- **[Freiburg Smith Waterman Tutorial](http://rna.informatik.uni-freiburg.de/Teaching/index.jsp?toolName=Smith-Waterman)**

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>2 |</span></b> <b>Class Stucture & Outline</b></div>

#### <b><span style='color:#E888BB'>BIOPYLIB - SEQUENCE ALIGNMENT PART</span></b>

- As part of the <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">biopylib</mark></b> module, the following classes are incorporated
- The main class for **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">biological sequence alignment</mark>** operations:
> <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">MSA</mark></b> for **<span style='color:#E888BB'>multiple sequence alignment</span>** operations & <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">PSA</mark></b> for **<span style='color:#E888BB'>pairwise sequence alignment operations</span>** <br>
> <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">SUBM</mark></b> class needs to be initialised and passed onto whichever alignment approach is of interest

#### <b><span style='color:#E888BB'>CLASS CONTENT</span></b>

The structure isn't too complicated, however there are some classes which inherit other classes:

- <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">SUBM</mark></b> | Substitution matrix related class <br>
- <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ali_view</mark></b> | class for displaying alignment -> passed onto class <code>ALI</code>, <code>PSA</code> <br>
- <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ALI</mark></b> | class for storage, modification and display of alignments  -> requires <code>ali_view</code> class <br>
- <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">PSA</mark></b> | class for pairwise sequence alignment operations -> requires <code>SQ</code>,<code>ali_view</code> classes <br>
- <b><mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">MSA</mark></b> | class for multiple sequence alignment operations -> requires <code>PSA</code> <br>

### <b><span style='color:#E888BB'> 3.2 |</span> SUBM example usage </b>

- We can utlise the class by itself, calling the functions/methods outlined above:

In [4]:
# Read Substitution Matrix from File
subm = SUBM()
subm.read("/kaggle/input/bioinformatics/substitution_matrices/blosum62.mat")

# Get all unique characters
print(f'alphabet: {subm.abc}')

# Get Character Scores
print(f"score pair W & N : {subm.score_pair('W','N')}") # Two Characters aren't identical
print(f"score pair W & N : {subm.score_pair('W','W')}") # Two Characters are identical
print(f"direct score pair W & N : {subm['W','W']}")     # used in some class functions

# Make out own substitution matrix
subm = SUBM()
subm.make('ATCG',2,-1)   # Constant Match Score of 2, mismatch score of -1 (all alphabet)
subm.head()

alphabet: CSTPAGNDEQHRKMILVFYW
score pair W & N : -4
score pair W & N : 11
direct score pair W & N : 11


[('AA', 2), ('AT', -1), ('AC', -1), ('AG', -1), ('TA', -1)]

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>4 |</span></b> <b>Sequence Alignment</b></div>

#### <b><span style='color:#E888BB'>USING BOKEH FOR ALIGNMENT VISUALISATION</span></b>

- Thanks to **[@nwheeler443](https://www.kaggle.com/nwheeler443)'s** **[Understanding Phylogenetic Trees](https://www.kaggle.com/nwheeler443/understanding-phylogenetic-trees)** notebook
- I've added the Bokeh function into class format (<code>ali_view</code>) & used simple **class inheritence** to add the option to <b>plot class alignments</b> for both the custom class & Biopython
- Both Biopython & Bioconductor lack such an awesome feature bokeh gives us, doable with plotly as well (although drains more resources).
> - So this is an example where we benefit from using custom classes, making our own modifications as we like
> - Over existing libraries as we can **immediately integrate them into our alignment classes code** & only call <code>.view</code>
- We can quickly **view very long alignmnents via scrolling**, which is much better printed alignments using a simple operation <code>.view()</code> after we have aligned sequences for <b>MSA</b> & <b>PSA</b>

#### <b><span style='color:#E888BB'>COLOUR SCHEMES</span></b>
- Although not quite as sophisticated as say **<span style='color:#E888BB'>bioconductor</span>**, **<span style='color:#E888BB'>bokeh</span>** does allow us to customise the background rectangular colours, so we can define our own colourcoding scheme
- Custom colourcoding schemes can be added into the static method **<span style='color:#E888BB'>get_colors</span>**, at the moment there are two options:
> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">general</mark>** : Very general colouring of characters
> - **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">charge_aa</mark>** : Colour variation of amino acids based on a simple division **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Hydrophobic</mark>** **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Negative</mark>** & **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Positive</mark>**, more information about **[hydrophobic & polar](https://www2.chem.wisc.edu/deptfiles/genchem/netorial/modules/biomolecules/modules/protein1/prot13.htm)** amino acids
- You can add your own colour scheme by adding a new dictionary named <code>clrs</code>. It makes sense set colours that can easily reveal patterns in the current sequence

### <b><span style='color:#E888BB'> 4.1 |</span> Alignment Visualisation Class </b>

Next, we'll be needing a class that can be used to **<span style='color:#E888BB'>visualise alignment</span>**, so multiple sequence lines:


**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ATTRIBUTES</mark>**

- <code>aln</code> - Stores the alignment
- <code>colcod</code> - Colourcoding options

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">METHODS</mark>**

- <code>get_colors</code> - method used to get the **<span style='color:#E888BB'>colour</span>** corresponding to the character (from a predefined dictionary)
- <code>view</code> - Create <b>Bokeh</b> gridplot, containing alignments

### <b><span style='color:#E888BB'> 4.2 |</span> Alignment Storage/Operations Class </b>

Next, we'll need a class for **<span style='color:#E888BB'>storing the alignment</span>** & any subsequent modiciation to the alignment (such as finding the **<span style='color:#E888BB'>consensus</span>**)

#### **<span style='color:#E888BB'>STORING & MANAGING ALIGNMENTS</span>**

- Having aligned two or more sequences using <b><span style='color:#E888BB'>pairwise or multiple</span></b> sequence alignment, we need a way to store this alignment data
- Similar to how we we can store <b><span style='color:#E888BB'>single sequences</span></b> in the <code>SQ</code> class, we need to be able to <b><span style='color:#E888BB'>store sequences which are alligned</span></b> as follows:
> **<span style='color:#E888BB'>[SEQUENCE 1]</span>** - ACG**GG**T-**ACG**TTT**C**A <br>
> **<span style='color:#E888BB'>[SEQUENCE 2]</span>** - GT**GG**---**ACG**GCC**C**T <br>
> ...
- Using class <code>ALI</code>, we are able to store the alignment information of two or multiple sequence and do various alignment related operations 
- The main operation that we need to use with <code>ALI</code> is <code>consensus</code>; which is used to find the most frequently occuring set of character in each column

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ATTRIBUTES</mark>**

- <code>lst_seq</code> - list of aligned sequences
- <code>al_type</code> - alignment sequence type
- <code>lst_SQ</code> - list of SQ objects of each sequence in the alignment
    
**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">METHODS</mark>**
    
- <code>__str__</code> - Special method, when using <code>print()</code>, basic sequence print & information
- <code>__len__</code> - Special method, when calling <code>len()</code>, shows the alignment length (should all be same)
- <code>col</code> - Show column number **n**'s characters
- <code>consensus</code> - Find the consesnsus between sequences
- <code>view</code> - View Alignment using Bokeh

### <b><span style='color:#E888BB'> 4.3 |</span> ALI Examples Usage </b>

- Although it is a class used for **storing stacked sequences after alignment** & used as a secondary class with MSA,PSA
- We can show examples of its use using **already aligned** <code>seq1</code>, <code>seq2</code> & <code>seq3</code>

In [5]:
# Having aligned three protein sequences we get 
seq1 = "MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW"
seq2 = "MH--IFIYQIGYALKSGYIQSIRSPEY-NW"
seq3 = "MHQAIFI-QIGYALKSGY-QSIRSPEYDNW"

# define an alignment (done during psa,msa)
alig2 = ALI(lst_seqs = [seq1,seq2,seq3],
             al_type = 'aa')

print(f'Return one of the alignments: {alig2[0]}\n')
print(f'Return alignmnent column {alig2.col(0)}\n')

# print alignment via print
print(alig2)

print(f'Consensus sequence: {alig2.consensus()}')

print(f'\nAlignment visualisation w/ bokeh')
alig2.view()

Return one of the alignments: MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW

Return alignmnent column ['M', 'M', 'M']

Alignment with 3 rows and 30 columns.
MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW
MH--IFIYQIGYALKSGYIQSIRSPEY-NW
MHQAIFI-QIGYALKSGY-QSIRSPEYDNW

Consensus sequence: MHQAIFIYQIGYALKSGYIQSIRSPEYDNW

Alignment visualisation w/ bokeh


Bokeh(Column)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>5 |</span></b> <b>Pairwise Sequence Alignment</b></div>

### <b><span style='color:#E888BB'> 5.1 |</span> Types of Pairwise Sequence Alignments </b>

- Two main classes of sequence alignments exist, **<span style='color:#E888BB'>global</span>** & **<span style='color:#E888BB'>local sequence alignment</span>**, as mentioned in **<span style='color:#E888BB'>Section 1.5</span>**
- A **[Dynamic Programming (DP)](https://developer.ibm.com/articles/j-seqalign/)** approach is often used for this problem, both algorithms mentioned here use this approach & it is also used in the <code>pairwise2</code> module of **[biopython](https://biopython.org/docs/1.75/api/Bio.pairwise2.html)**
- All algorithms aim to achieve the same thing & differ mainly in speed of processing, DP is generally not used for a very sizable number of sequences, however two sequences are more than fine

### **<span style='color:#E888BB'>GLOBAL</span> SEQUENCE ALIGNMENT**

**<span style='color:#E888BB'>THE AIM</span>** | We want to **<span style='color:#E888BB'>align the two provided sequences completely (entire length)</span>**
> - It is therefore best for highly similar sequences of **approximately the same length**
> - Popular global sequence alignment algorithm; **<span style='color:#E888BB'>Needleman-Wunsch</span>**

### **<span style='color:#E888BB'>LOCAL </span> SEQUENCE ALIGNMENT**
**<span style='color:#E888BB'>THE AIM </span>** | We want to **<span style='color:#E888BB'>find a part of both segments</span>**, which **<span style='color:#E888BB'>has the best alignment</span>**
> - Local alignment **<span style='color:#E888BB'>searches for the most similar regions</span>** in the two sequences
> - It is best used for sequences that share some degree of similarity or of different lengths
> - Popular local sequence alignment algorithm; **<span style='color:#E888BB'>Smith-Waterman</span>**

### <b><span style='color:#E888BB'> 5.2 |</span> Needleman-Wunsch Algorithm </b>

### **<span style='color:#E888BB'>STEPS IN THE ALGORITHM</span>**

**<span style='color:#E888BB'>Needleman-Wunsch Algorithm</span>** consists of three steps:
> - (1) Initialisation of the **<span style='color:#E888BB'>Score Matrix</span>** (<code>.SM</code>)
> - (2) Calculation of the scores and filling of the **<span style='color:#E888BB'>Traceback Matrix</span>** (<code>.TM</code>)
> - (3) **<span style='color:#E888BB'>Realignment</span>** | Deducing the alignment from the Traceback Matrix (<code>.TM</code>)

### **RELATING TO <span style='color:#E888BB'> SCORING MATRIX (SM)</span>**

- The purpose of the algorithm is to **<span style='color:#E888BB'>fill the score matrix</span>**, <code>.SM</code> 
- The main idea of the algorithm is that this matrix <code>.SM</code> can be **<span style='color:#E888BB'>filled cell by cell</span>**, **<span style='color:#E888BB'>using adjacent cells to reach the value of the target cell</span>**.
- Initialising the top row & left column, we have an **<span style='color:#E888BB'>initial condition</span>** & move towards the bottom right corner, which is our target value.
> - For global alignment, the initialisation values are set to g x n (number of columns in the alignment x gap)

**<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">SCORE MATRIX</mark>** | The reccurence relation used to fill the scoring matrix, moving from top left to bottom right:

> - v1 = S[i-1,j-1] + sm(a[i]b[j])
> - v2 = S_[i-1,j] + g
> - v3 = S_[i,j-1] + g
> -  S[i][j] = max(v1,v2,v3)

### **RELATING TO <span style='color:#E888BB'>TRACE-BACK MATRIX (TM)</span>**
- Encoding a <b>three direction constants/alphabets</b>, we want to also **<span style='color:#E888BB'>keep track of our alignment pathways</span>** we take when filling out the <b>SM</b>
- The information is stored in the **<span style='color:#E888BB'>traceback matrix</span>**, as we are scoring. 

### <b><span style='color:#E888BB'> 5.3 |</span> Smith-Waterman Algorithm</b>

- We are interested in the best subset alignment among the two sequences of interest, that **<span style='color:#E888BB'>maximises the objective function</span>**, ie. score
- The **<span style='color:#E888BB'>algorithm consists of the same three steps</span>** as outied above:
> - (1) Initialisation of the **<span style='color:#E888BB'>Score Matrix</span>** (<code>.SM</code>)
> - (2) Calculation of the scores and filling of the **<span style='color:#E888BB'>Traceback Matrix</span>** (<code>.TM</code>)
> - (3) **<span style='color:#E888BB'>Realignment</span>** | Deducing the alignment from the Traceback Matrix (<code>.TM</code>)
- The main idea of the algorithm is that this matrix <code>.SM</code> can be **<span style='color:#E888BB'>filled cell by cell</span>**, **<span style='color:#E888BB'>using adjacent cells to reach the value of the target cell</span>**, similar to the algorithm above.

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SCORE MATRIX</mark>** | The reccurence relation used to fill the scoring matrix, moving from top left to bottom right:

> - v1 = S[i-1,j-1] + SM(a[i],b[j])
> - v2 = S[i-1,j] + g
> - v3 = S[i,j-1] + g
> - S[i][j] = max(v1,v2,v3,0)

- If the best alternative from the three considered before leads to a negative score, this means that in that position the best is to restart the alignment from this position onward, ignoring the previous parts of the sequence
- Initialising the top row & left column, we have an **<span style='color:#E888BB'>initial condition</span>** & move towards the bottom right corner, which is our target value.
> - For local alignment the initialisation values are set to 0

### <b><span style='color:#E888BB'> 5.4 |</span> Pairwise Sequence Alignment Class </b>

- Having defined the information about **<span style='color:#E888BB'>pairwise</span>** sequence alignment above, we can define the main class:

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ATTRIBUTES</mark>**

- <code>seq1</code> - Storage of sequence #1 (<code>SQ</code> class) 
- <code>seq2</code> - Storage of sequence #2 (<code>SQ</code> class) 
- <code>g</code> - Input requires storage of a gap penalty (constant value)
- <code>subm</code> - Input requires storage of a substitution matrix (needs to be set before)

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">METHODS</mark>**

- <code>maxidx</code> - Find index of max value
- <code>score_pos</code> - If a gap was not defined, set the score value from 
- <code>initial_condition</code> - Set initial scoring & traceback matrix conditions
- <code>nW</code> - Calculate scoring & traceback matrices
- <code>realign</code> - Realign sequence from the traceback matrix & store in <code>ALI</code> class
- <code>view</code> - View pairwise sequence alignment

### <b><span style='color:#E888BB'> 5.5 |</span> Global Alignment Walkthrough </b>

- As mentioned above, the algorithm revolves around filling two matrices **<span style='color:#E888BB'>scoring & traceback</span>** matrices
- To start iterating from the top left corner to the bottom right, we need to initialise the top-left corner of both matrices

### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">NEEDLEMAN-WUNSCH</mark>**  **| INITIAL CONDITION EXAMPLE**

- As stated above, the algorithm requires some starting condition, which involves the use of the <code>g</code> values.
- In the Needleman-Wunsch case, we initialise the row & column by 2x<code>g</code> x n except the top left corner.

In [6]:
seq3 = SQ(seq="KPFKRTCYKPF",seq_type='aa')
seq4 = SQ(seq="AKPKKPYI",seq_type='aa')

# Define Substitution Matrix 
blosum62 = '/kaggle/input/bioinformatics/substitution_matrices/blosum62.mat'

# # Global Sequence Alignment
gpsa_prot = PSA(seqs=[seq3,seq4],
                subm=SUBM().read(blosum62),
                g=-2)     

gpsa_prot.initial_condition(ids='nW') # by default included in .nW method 

### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">NEEDLEMAN-WUNSCH</mark>**  **| FILLED SM & TM MATRICES**
- As per the scoring rules outlined in **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Section 5.3</mark>**, we obtain the filled scoring matrix, for the global alignment
- We're only interested in the value at the bottom right hand corner; **<span style='color:#E888BB'>global alignment score</span>**

In [7]:
sm = SUBM()
sm.read(blosum62)

gpsa_prot = PSA(seqs=[seq3,seq4],
                subm=sm,
                g=-2)    

gpsa_prot.nW()
print('Finalised Score Matrix')

Finalised Score Matrix


In [8]:
print(f'global alignment score: {gpsa_prot.score}')

global alignment score: 7


In [9]:
# Traceback Movement interpretation/mapping
# 0 - done, 1 - diagonal, 2 - up, 3 - left
gpsa_prot.pprint(ids='TM')

,gap,A,K,P,K,K,P,Y,I
gap,0,3,3,3,3,3,3,3,3
K,2,1,1,3,3,3,3,3,3
P,2,2,2,1,3,3,3,3,3
F,2,2,2,2,1,3,3,1,3
K,2,2,1,3,1,1,3,3,3
R,2,2,2,1,2,1,3,3,3
T,2,1,2,1,2,2,1,3,3
C,2,2,2,2,2,2,2,1,1
Y,2,2,2,2,2,2,2,1,3
K,2,2,1,2,1,1,3,2,1


 ### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">NEEDLEMAN-WUNSCH</mark>**  **| RECONSTRUCTING ALIGNMENT**
 - In global alignment, to realign the two sequences, we start at the **<span style='color:#E888BB'>bottom right hand corner</span>** (the global score, which was 7)
 - In literature, typically you'd find arrows as an indicator of how to realign, but **<span style='color:#E888BB'>uniquely set numbers</span>** work as well:
 > 0 - done, 1 - diagonal, 2 - up, 3 - left
 - The alignment is **<span style='color:#E888BB'>built in reverse order</span>**, as we start at the very end corner, so here are the simple rules:
 > - If we move diagonally (1), we set the column to both of the character
 > - If we move vertically (up/2), the column will contain the first sequence character & a gap in the second
 > - If we move horizontally (left/3), the column will contain the second sequence character & a gap in the first

In [10]:
realign = gpsa_prot.realign() # deduce alignment from TM matrix    
print(realign)

Alignment with 2 rows and 12 columns.
-KPFKRTCYKPF
AKP-KKP-Y--I



### <b><span style='color:#E888BB'> 5.6 |</span> Local Alignment Walkthrough </b>

- As with the **<span style='color:#E888BB'>needleman-wunsch</span>** alignment, the **<span style='color:#E888BB'>smith-waterman</span>** algorithm also revolves around filling two matrices **<span style='color:#E888BB'>scoring & traceback</span>** matrices
- To start iterating from the top left corner to the bottom right, we need to initialise the top-left corner of both matrices
- Some minor variation does exist between the two specified alignment algorithms: **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">NEEDLEMAN-WUNSCH</mark>** **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SMITH-WATERMAN</mark>**

### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SMITH-WATERMAN</mark>**  **| INITIAL CONDITION EXAMPLE**
- In the Smith-Waterman case, the matrix top left side is initialised with 0 as shown in **[this reference](https://vlab.amrita.edu/?sub=3&brch=274&sim=1433&cnt=1)**

In [11]:
seq3 = SQ(seq="HGWAG",seq_type='aa')
seq4 = SQ(seq="PHSWG",seq_type='aa')

sm = SUBM()
sm.read(blosum62)

lpsa_prot = PSA(seqs=[seq3,seq4],
                subm=sm,
                g=-2)    

lpsa_prot.initial_condition(ids='sW')
lpsa_prot.pprint(ids='SM')

,gap,P,H,S,W,G
gap,0,0.0,0.0,0.0,0.0,0.0
H,0,NaN,NaN,NaN,NaN,NaN
G,0,NaN,NaN,NaN,NaN,NaN
W,0,NaN,NaN,NaN,NaN,NaN
A,0,NaN,NaN,NaN,NaN,NaN
G,0,NaN,NaN,NaN,NaN,NaN


### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SMITH-WATERMAN</mark>**  **| FILLED SM & TM MATRICES**

- The matrix size of <code>.SM</code> & <code>.TM</code> is proportional to the sizes of the input sequences <code>.seq1</code> & <code>.seq2</code>
- In the example below, we have a symmetric matrix, as both sequences are of identical length
- We can also see the usage order of all class methods below
> - In the example below, we'll locally align two sequences, <code>KPFKRTCYKPF</code> & <code>AKPKKPYI</code>
> - And fill the matrices using the <code>sW()</code> method, after which the matrices can be called using <code>.SM</code> & <code>.TM</code>, or shown via the <code>pprint</code> method
- The best score turns out to be 7, and we have a few cases where we have a maximum, however no multiple case scenario is taken into account in the code

In [12]:
seq3 = SQ(seq="KPFKRTCYKPF",seq_type='aa')
seq4 = SQ(seq="AKPKKPYI",seq_type='aa')

sm = SUBM()
sm.read(blosum62)

# # Global Sequence Alignment
lpsa_prot = PSA(seqs=[seq3,seq4],
                subm=sm,
                g=-8)     
lpsa_prot.sW()
print('Finalised Score Matrix')
lpsa_prot.pprint(ids='SM')
print(f'\nlength of symmetric matrix {len(lpsa_prot.SM)}')
print(f'best alignment score: {lpsa_prot.score}')

Finalised Score Matrix


,gap,A,K,P,K,K,P,Y,I
gap,0,0,0,0,0,0,0,0,0
K,0,0,5,0,5,5,0,0,0
P,0,0,0,4,0,3,4,0,0
F,0,0,0,0,1,0,0,7,0
K,0,0,5,0,5,6,0,0,4
R,0,0,2,3,2,7,4,0,0
T,0,0,0,1,2,1,6,2,0
C,0,0,0,0,0,0,0,4,1
Y,0,0,0,0,0,0,0,7,3
K,0,0,5,0,5,5,0,0,4



length of symmetric matrix 12
best alignment score: 7


In [13]:
# Traceback Movement interpretation/mapping
# 0 - done, 1 - diagonal, 2 - up, 3 - left
lpsa_prot.pprint(ids='TM')

,gap,A,K,P,K,K,P,Y,I
gap,0,0,0,0,0,0,0,0,0
K,0,0,1,0,1,1,0,0,0
P,0,0,0,1,0,1,1,0,0
F,0,0,0,0,1,0,0,1,0
K,0,0,1,0,1,1,0,0,1
R,0,0,1,1,1,1,1,0,0
T,0,0,0,1,1,1,1,1,0
C,0,0,0,0,0,0,0,1,1
Y,0,0,0,0,0,0,0,1,1
K,0,0,1,0,1,1,0,0,1


 ### **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SMITH-WATERMAN</mark>**  **| RECONSTRUCTING ALIGNMENT**

- The best alignment can occur in any cell of the score/traceback matrix, unlike in the global alignment case
- Searching for the maximum value in the score matrix, we can note that we have only one best alignment, with a score of 4 (column GA)
- Taking into account local alignment, **<span style='color:#E888BB'>uniquely set numbers</span>** work as well:
 > 0 - terminate alignment, 1 - diagonal, 2 - up, 3 - left
- The alignment is **<span style='color:#E888BB'>built in reverse order</span>**, as we start at the maximum score value, so here are the simple rules:
 > - If we move diagonally (1), we set the column to both of the character
 > - If we move vertically (up/2), the column will contain the first sequence character & a gap in the second
 > - If we move horizontally (left/3), the column will contain the second sequence character & a gap in the first
 > - If we end up at a value of 0, the alignment is terminated

In [14]:
realign = lpsa_prot.realign(ids='sW') # deduce alignment from TM matrix    
print(realign) # print ALN class object

Alignment with 2 rows and 3 columns.
KPF
KPY



### <b><span style='color:#E888BB'> 5.7 |</span> Visualise Alignment </b>

- So we've filled the **<span style='color:#E888BB'>scoring (SM) & traceback matrix (TM)</span>**
- Used the **<span style='color:#E888BB'>traceback matrix</span>** to **<span style='color:#E888BB'>reconstruct</span>** the sequence alignment
- Now want to visualise the alignment (**<span style='color:#E888BB'>ALN</span>** object), to do that we can call the <code>.view()</code> method

In [15]:
# Nucleotide Sequence Example (Doesn't have to be identical length)
seq1 = SQ(seq="tccCAGATATGTCAGGGGACACGAGcatgcagagac",seq_type='dna')
seq2 = SQ(seq="CATCATCATCATCATCATCATCATCATCAT",seq_type='dna')

# Create a substitute matrix (nucleotides)
sm = SUBM()
sm.make("ACGT",1,0)    # match +1 / mismatch -1
g = -2                 # gap penalty -2

# Global PSA alignment (get score only)
gpsa_nucl = PSA(seqs=[seq1,seq2],
                subm=sm,
                g=g)     

gpsa_nucl.nW() # calculate score, SM & TM
realign = gpsa_nucl.realign() # deduce alignment from TM matrix     

In [16]:
gpsa_nucl.view() # visualise alignment

Bokeh(Column)

### <b><span style='color:#E888BB'> 5.8 |</span> Pairwise Sequence Alignment Example </b>

####  <b><span style='color:#E888BB'>HUMAN & MOUSE PROTEIN SEQUENCE ALIGNMENT</span></b> 
- So we know how to read realistic sequences from **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">FASTA</mark>** files, so let's look at a more realistic example
- Now let’s align the **<span style='color:#E888BB'>human</span>** (<code>AAH12844.1.faa</code>) and **<span style='color:#E888BB'>mouse</span>** (<code>NP_001265185.1.faa</code>) major prion proteins typing their accession numbers
- We'll also define a custom colorcoding palette **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">chage_aa</mark>**, custom palettes can be added using **<span style='color:#E888BB'>@staticmethod</span>** <code>get_colors</code> in class **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ali_view</mark>** 

In [17]:
human = "/kaggle/input/bioinformatics/sequences/AAH12844.1.faa"      
mouse = "/kaggle/input/bioinformatics/sequences/NP_001265185.1.faa"

# Read sequences from FASTA files
human_faa = readSeq(human).store(); print(f'length of sequence: {len(human_faa)}')
mouse_faa = readSeq(mouse).store(); print(f'length of sequence: {len(mouse_faa)}')

[note] read -> FASTA [amino acid] | #seq: 1
length of sequence: 253
[note] read -> FASTA [amino acid] | #seq: 1
length of sequence: 254


In [18]:
# Instantiate substitution matrix
subm62 = SUBM()
subm62.read("/kaggle/input/bioinformatics/substitution_matrices/blosum62.mat")

alin = PSA(seqs=[human_faa,mouse_faa], # define sequences
           subm=subm62,g=-3,         # use BLOSUM62 substitution matrix
           colcod = 'charge_aa') # use colourcoding in view

alin.nW() # calculte SM, TM matrices 
alin.realign(ids='nW') # get the alignment from TM matrix
alin.view() # visualise alignment

Bokeh(Column)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>6 |</span></b> <b>Multiple Sequence Alignment</b></div>

### <b><span style='color:#E888BB'> 6.1 |</span> Progressive Alignment Approach </b>

- We looked at **<span style='color:#E888BB'>PSA</span>**, now we'll focus on utilising it for **<span style='color:#E888BB'>multiple sequence alignment</span>**:
- The method first calls **<span style='color:#E888BB'>PSA</span>**, then progressively, subsequent sequences are added and a consensus is constantly found iteratively
- Unlike **<span style='color:#E888BB'>PSA</span>**, here we'll only be concerning ourselves with **<span style='color:#E888BB'>global sequence alignment</span>**

#### **<span style='color:#E888BB'>ITERATIVELY ADD SEQUENCES</span>** - **PROGRESSIVE APPROACH** 
- Various approaches exists for **<span style='color:#E888BB'>MSA</span>**, one such method is called the **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">progressive algorithm</mark>** apprach
- It's quite a straightforward iterative approach, however it lacks scalability for large number of sequences 
- Usually replaced by more advanced methods like **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ClustalW</mark>**, **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ClustalOmega</mark>**, and **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">Muscle</mark>**, which are incorporated into libraries like **<span style='color:#E888BB'>Bioconductor</span>**

In the **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">progressive algorithm</mark>** approach:
- **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">STEP 1</mark>** We **<span style='color:#E888BB'>start with two sequences</span>** from a list of **2+ sequences** & first align the two globally with PSA
- Subseqent steps will involve repeating the following for each addition sequence after the pair:
> - **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">STEP 2</mark>** Determining a **<span style='color:#E888BB'>consensus between the two sequences that were aligned</span>**; creating the **first sequence** in subsequent PSA operations
> - For the **<span style='color:#E888BB'>second sequence</span>**, we add the next sequence in the list, thus we have two sequences again (REPEAT STEP1)

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">CONSENSUS</mark><span style='color:#5D2ECC'></span>** | **[SCIENCEDIRECT](https://www.sciencedirect.com/topics/medicine-and-dentistry/consensus-sequence)**

> - A consensus sequence is a sequence of DNA, RNA, or protein that represents aligned, related sequences
> - The consensus sequence of the related sequences can be defined in different ways, but is normally defined by the most common nucleotide(s) or amino acid residue(s) at each position

### <b><span style='color:#E888BB'> 6.2 |</span> Multiple Sequence Alignment </b>

- The method for progressive approach to **<span style='color:#E888BB'>MSA</span>** was described above, now let's show the actual class that we'll be using

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">CONSTRUCTOR</mark>**

- <code>seqs</code> - The constructor stores the list of sequence (SQ) objects
- <code>subm</code> - The substitution matrix object/instance
- <code>g</code> - The gap penalty model (constant value)
- <code>colcod</code> - The colourcoding of each character in the visualisation <code>.view()</code>

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">GENERAL CLASS METHODS</mark>**
- <code>num_seqs</code> - Return the number of sequences in the alignment
- <code>add_ali</code> - Helper function, used by other functions in order to add an alignment to the existing alignment
- <code>align_consensus</code> - Main progressive MSA method/function
- <code>view</code> - view MSA alignment

### <b><span style='color:#E888BB'> 6.3 |</span> Examples </b>

In [19]:
# Define some sequences
s1 = SQ("ATAGCACTCATCCGGCCAG")
s2 = SQ("AACCCTGCAACATAGAGCA")
s3 = SQ("ATGACCATGGAGACCCTGA")
s4 = SQ("ATGGCCCATGGGCGTACTG")
s5 = SQ("ATGGCAACTACGGGACACC")

# Like in PSA, set a substitution matrix
sm = SUBM()
sm.make("ACGT",1,-1)

# Initialisation of MSA problem
ma = MSA(seqs=[s1,s2,s3,s4,s5], # set the sequences 
         subm=sm,               # set the substitution matrix
         g=-1)                  # set the gap penalty model

# Get the alignment
al = ma.align_consensus() # Iteratively align sequences
al.view() # visualise alignment 

Bokeh(Column)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>7 |</span></b> <b>BioPython Sequence Alignment</b></div>

### <b><span style='color:#E888BB'> 7.1 |</span> Popular Bioinformatics Libraries </b>

There are two libraries which are quite popular & accessable on Kaggle & both contain **<span style='color:#E888BB'>sequence alignment</span>** libraries:
> - **<span style='color:#E888BB'>BioPython</span>** **(Python)** | Shown in **[Biopython | Bioinformatics Basics](https://www.kaggle.com/shtrausslearning/biopython-bioinformatics-basics)**:
    - **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Pairwise Sequence Alignment</mark>** is obtained using <code>pairwise2</code>
    - **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Multiple Sequence Alignment</mark>** is obtained using the <code>Align.MultipleSequenceAlignment</code>
    
    
> - **<span style='color:#E888BB'>Bioconductor</span>** **(R)** - Another library that contains both:
    -  **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Pairwise Sequence Alignment</mark>** found in the <code>Biostrings</code> library
    -  **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">Multiple Sequence Alignment</mark>** found in the <code>msa</code> library
    
The notebook being Python based, we'll look at the former & look at the latter in a separate notebook - **[Bioconductor | Bioinformatics Basics](https://www.kaggle.com/shtrausslearning/bioconductor-bioinformatics-basics)**

### <b><span style='color:#E888BB'> 7.2 |</span> Sequence classes </b>
    
#### **<span style='color:#E888BB'>STORING SEQUENCES</span>**    
The two main classes used for **<span style='color:#E888BB'>sequence</span>** storage in Biopython: 
> - <code>Seq</code> - Basic sequence storage class
> - <code>SeqRecord</code> - More advanced sequence storage class, can store **<span style='color:#E888BB'>alignment</span>** as well ( require <code>Seq</code> class objects )

In [20]:
from Bio.pairwise2 import format_alignment  # formatted pairwise display 
from Bio import pairwise2 # pairwise sequece alignment
from Bio.Seq import Seq # basic sequence class
from Bio.SeqRecord import SeqRecord # more detailed sequence class 

# Basic String Format
str_seq = 'ACAAATTCAATTCCATAACATTTATCCCAT'
basic_seq = Seq(str_seq)
print(basic_seq,'\n')

ACAAATTCAATTCCATAACATTTATCCCAT 



In [21]:
# Advanced String Format
print(type(basic_seq))
adv_seq = SeqRecord(basic_seq,id='basic_seq')
print(adv_seq)
print(type(adv_seq))

<class 'Bio.Seq.Seq'>
ID: basic_seq
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('ACAAATTCAATTCCATAACATTTATCCCAT')
<class 'Bio.SeqRecord.SeqRecord'>


### <b><span style='color:#E888BB'> 7.3 |</span> Pairwise Sequence Alignment </b>
    
#### **<span style='color:#E888BB'>MODULES - </span>ALIGNING SEQUENCES**  
For BioPython **<span style='color:#E888BB'>Sequence Alignment</span>**:
> - <code>Seq</code> & <code>string</code> formats can be used as inputs in <code>pairwise2</code>, 
> - However for <code>MultipleSeqAlignment</code>, we need to input sequences in the form of <code>SeqRecord</code> objects.

#### <b><span style='color:#E888BB'>PAIRWISE SEQUENCE ALIGNMENT</span> - PAIRWISE2</b> 
- **<span style='color:#E888BB'>BioPython documentation</span>** | **[pairwise2](https://biopython.org/docs/1.75/api/Bio.pairwise2.html)**
- Let's align two sequences <code>cseq1</code> & <code>cseq2</code> using a global alignment approach, we can input either <b>string sequences</b> or <b>SQ object sequences</b>
- We can visualise the **region which has lots of gaps** (perhaps from **mutation**) & the **section which aligns perfectly**
- We are looking at **<span style='color:#E888BB'>global alignment</span>** (<code>.global</code>) of these two sequences:
> - with a **<span style='color:#E888BB'>match score of 1</span>** & **<span style='color:#E888BB'>missmatch score of 0</span>** (so **first letter is x**) & **<span style='color:#E888BB'>no gap penalty</span>** emposed (**second letter is x**)
- We can note that for this example, we have more than one possible outcome for the same score as well

### **POST ALIGNMENT**

- Upon obtaining the alignment, results are of <code>Bio.pairwise2.Alignment</code> class type (<b>if there are no ties</b>)
- Otherwise a list with the same class type is created for **all the possible alignmnent combinations**
- <code>.format_alignment()</code> - used for sequence alignment visualisation (show <b>full_sequences</b>=True)

<b><span style='color:#E888BB'>MORE EXAMPLES</span></b> | If you are interested in more detailed examples, you can check out the notebook **[BioPython | Bioinformatics Basics](https://www.kaggle.com/shtrausslearning/biopython-bioinformatics-basics)**

In [22]:
''' Global Alignment of two DNA sequences'''

# define an instance of Seq
cseq1 = Seq('ATGGCAGATAGA')
cseq2 = Seq('ATAGAGAATAGATGGCAGATAGA')

# - (match score = 1, missmatch = 0), gap penalties = 0
GDNA = pairwise2.align.globalxx(cseq1,cseq2)
print(f'PSA class: {type(GDNA[0])}')
print (f'# Alternative optimal alignments: {len(GDNA)}\n')
print('Aligned Sequence #0')
print(GDNA[0])

print('\nShowing all score ties:')

# print all the alignments (including ties)
for i in GDNA: 
    print(format_alignment(*i,full_sequences=True))

PSA class: <class 'Bio.pairwise2.Alignment'>
# Alternative optimal alignments: 11

Aligned Sequence #0
Alignment(seqA='AT-G-G---------CAGATAGA', seqB='ATAGAGAATAGATGGCAGATAGA', score=12.0, start=0, end=23)

Showing all score ties:
AT-G-G---------CAGATAGA
|| | |         ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

AT-G------G----CAGATAGA
|| |      |    ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

AT---G----G----CAGATAGA
||   |    |    ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

AT-G---------G-CAGATAGA
|| |         | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

AT---G-------G-CAGATAGA
||   |       | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

AT--------G--G-CAGATAGA
||        |  | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

A-------T-G--G-CAGATAGA
|       | |  | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

--A-----T-G--G-CAGATAGA
  |     | |  | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

----A---T-G--G-CAGATAGA
    |   | |  | ||||||||
ATAGAGAATAGATGGCAGATAGA
  Score=12

------A-T-G--

### <b><span style='color:#E888BB'> 7.4 |</span> Multiple Sequence Alignment </b>

#### <b><span style='color:#E888BB'>MULTIPLE SEQUENCE ALIGNMENT</span> - MultipleSeqAlignment</b> 
- **<span style='color:#E888BB'>BioPython documentation</span>** | **[MultipleSequenceAlignment](https://biopython.org/docs/1.76/api/Bio.Align.html)** (Class part of <code>Bio.Align</code> class)
- Using this class, we can align **<span style='color:#E888BB'>nucleotide</span>** & **<span style='color:#E888BB'>amino acid sequences</span>**, with or without <b>gaps</b> present.
- To align <b><span style='color:#E888BB'>Multiple Sequences</span></b> together ( <b>More than two</b> ), we can use the <code>.Align</code> class & call <code>MultipleSeqAlignment</code>
- The alignment can be viewed using the **<mark style="background-color:#E888BB;color:white;border-radius:5px;opacity:0.9">ali_view</mark>** class as well

#### <b><span style='color:#E888BB'>USING SEQRECORD SEQUENCE FORMAT</span></b>

- If we define sequences from <b>strings</b> we'll need to define an <code>Seq</code> instance & subsequently <code>SeqRecord</code> (only needing <code>id</code>)
- However unlike <code>Bio.pairwise2</code>, we can't use direct <b>strings</b> in the alignment class input & need to define a <code>Seq</code> object for a particular sequence.
- Followed by a defined <b>SeqRecord</b> instance that has its defined <b>sequence ID</b>.

In [23]:
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment as MSABIO

# String format of alignment
aa1 = "MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW"
aa2 = "MH__IFIYQIGYALKSGYIQSIRSPEY_NW"
aa3 = "MHQAIFI_QIGYALKSGY_QSIRSPEYDNW"
aa4 = "MHQAMHI_KSGYA__SGY_QSIRSPEYDNW"

# Sequence Class instances
seq_aa1 = Seq(aa1)
seq_aa2 = Seq(aa2)
seq_aa3 = Seq(aa3)
seq_aa4 = Seq(aa4)
 
# Create a sequence record w/ a defined ID
seqr1 = SeqRecord(seq_aa1,id="seq1")
seqr2 = SeqRecord(seq_aa2,id="seq2")
seqr3 = SeqRecord(seq_aa3,id="seq3")
seqr4 = SeqRecord(seq_aa4,id="seq4")
lst_seq = [seqr1,seqr2,seqr3,seqr4]

# Multiple Sequence Alignment
alin = MSABIO(lst_seq)
print(alin)

Alignment with 4 rows and 30 columns
MHQAIFIYQIGYPLKSGYIQSIRSPEYDNW seq1
MH__IFIYQIGYALKSGYIQSIRSPEY_NW seq2
MHQAIFI_QIGYALKSGY_QSIRSPEYDNW seq3
MHQAMHI_KSGYA__SGY_QSIRSPEYDNW seq4


# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#E888BB"><b><span style='color:#FFFFFF'>8 |</span></b> <b>IMPORTANT TAKEAWAYS</b></div>

#### <b><span style='color:#E888BB'>ALIGNING SEQUENCES USING THE BIOPYLIB MODULE</span> </b> 
- Examples of aligning **multiple sequences** in the **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">SQ</mark>** format using **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">PSA</mark>** & **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">MSA</mark>** & visualising the alignment using **<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">bokeh</mark>**

In [24]:
''' ALIGN TWO SEQUENCES '''

# Path to two amino acid chains
human = "/kaggle/input/bioinformatics/sequences/AAH12844.1.faa"      
mouse = "/kaggle/input/bioinformatics/sequences/NP_001265185.1.faa"

# Read sequences from FASTA files
human_faa = readSeq(human).store(); print(f'length of sequence: {len(human_faa)}')
mouse_faa = readSeq(mouse).store(); print(f'length of sequence: {len(mouse_faa)}')

# Instantiate substitution matrix
subm62 = SUBM()
subm62.read("/kaggle/input/bioinformatics/substitution_matrices/blosum62.mat")

# Pairwise Sequence Alignment
alin = PSA(seqs=[human_faa,mouse_faa], # define sequences
           subm=subm62,g=-3,         # use BLOSUM62 substitution matrix
           colcod = 'charge_aa') # use colourcoding in view

# Needleman Wunsch Global Alignment
alin.nW() # calculte SM, TM matrices 
alin.realign(ids='nW') # get the alignment from TM matrix
alin.view() # visualise alignment

[note] read -> FASTA [amino acid] | #seq: 1
length of sequence: 253
[note] read -> FASTA [amino acid] | #seq: 1
length of sequence: 254


Bokeh(Column)

In [25]:
''' ALIGN MULTIPLE SEQUENCES '''

# Define some sequences
s1 = SQ("ATAGCACTCATCCGGCCAG")
s2 = SQ("AACCCTGCAACATAGAGCA")
s3 = SQ("ATGACCATGGAGACCCTGA")
s4 = SQ("ATGGCCCATGGGCGTACTG")
s5 = SQ("ATGGCAACTACGGGACACC")

# Like in PSA, set a substitution matrix
sm = SUBM()
sm.make("ACGT",1,-1)

# Initialisation of MSA problem
ma = MSA(seqs=[s1,s2,s3,s4,s5], # set the sequences 
         subm=sm,               # set the substitution matrix
         g=-1)                  # set the gap penalty model

# Get the alignment
al = ma.align_consensus() # Iteratively align sequences
al.view() # visualise alignment 

Bokeh(Column)